# Predictive Maintenance: Turbofan Engine Remaining Useful Life (RUL) Prediction - Model Retraining/Fine-tuning

This notebook focuses on the **Re-Training** of an already deployed or pre-trained LSTM model for Remaining Useful Life (RUL) prediction. In real-world scenarios, models often need to be updated with new data or adapted to slightly different conditions without starting training from scratch. Fine-tuning allows us to leverage the knowledge gained by the pre-trained model and efficiently update it.

## Table of Contents

1.  [Setup and Data Loading](#1.-Setup-and-Data-Loading)
    * [1.1 Importing Libraries and Loading Preprocessed Data](#1.1-Importing-Libraries-and-Loading-Preprocessed-Data)
    * [1.2 Loading the Pre-trained Model and Scalers](#1.2-Loading-the-Pre-trained-Model-and-Scalers)
2.  [Model Re-Training](#2.-Model-Re-Training)
    * [2.1 Recompiling for Re-Training](#2.1-Recompiling-for-Re-Training)
    * [2.2 Re-Training the Model](#2.2-Re-Training-the-Model)
3.  [Saving the Re-Trained Model](#3.-Saving-the-Re-Trained-Model)
4.  [Conclusion](#4.-Conclusion)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from custom_functions import *
from custom_functions import PHM2008Score
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## 1. Setup and Data Loading

This section handles the necessary imports and loads the preprocessed data, along with the pre-trained model and scalers, preparing the environment for the fine-tuning process.

### 1.1 Importing Libraries and Loading Preprocessed Data

We begin by importing all essential libraries for model operations, including `tensorflow` for the deep learning model, `numpy` for numerical operations, `pandas` for data handling, and `joblib` for loading pre-saved scalers. Crucially, we also import `PHM2008Score` from `custom_functions`, which is a custom metric relevant to this predictive maintenance challenge.

The preprocessed `X_train` and `y_train` NumPy arrays, generated from the `EDA.ipynb` notebook, are loaded. These arrays represent the time-series sequences and their corresponding RUL values, ready for model training. The `rul_scaler` and `feature_scaler` are also loaded to ensure consistent data transformation.

### 1.2 Loading the Pre-trained Model and Scalers

Before fine-tuning, we load the pre-trained LSTM model, which was previously optimized (e.g., from `Model_Tuner.ipynb`). This model has already learned general degradation patterns from the initial training data. Fine-tuning will allow us to adapt this model to potentially new data or refine its performance without rebuilding from scratch.

The output confirms that the pre-trained model has been successfully loaded. This model will now serve as the starting point for the retraining process.

In [ ]:
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')

rul_scaler = joblib.load('rul_scaler.pkl')
scaler = joblib.load('feature_scaler.pkl')

In [ ]:
# --- Load Your Deployed Model and Scalers ---
model_save_path_keras = "final_lstm_model.keras" # Path to your deployed model
fine_tune_model = tf.keras.models.load_model(model_save_path_keras)
print("Pre-trained model loaded successfully!")

In [ ]:
# --- Fine-tune the Model ---
print("\n--- Starting Fine-tuning Training ---")

# Recompile the model with a much smaller learning rate
# This prevents drastic changes to already learned weights
new_learning_rate = 0.0000436 # A common practice: use 1/10th or 1/100th of your original LR (Here its 1/10)
fine_tune_model.compile(optimizer=Adam(learning_rate=new_learning_rate),
                        loss='mae',
                        metrics=['mae', PHM2008Score()])

# Callbacks for fine-tuning

# Create a callback to stop training when a metric has stopped improving
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_phm_2008_score', patience=10, mode='min', restore_best_weights=True)

fine_tuned_model_filepath = 'fine_tuned_lstm_model_v2.keras' # New filename for fine-tuned model
model_checkpoint_ft = ModelCheckpoint(
    filepath=fine_tuned_model_filepath,
    monitor='val_phm_2008_score',
    save_best_only=True,
    mode='min',
    verbose=0
)

callbacks_list_ft = [stop_early, model_checkpoint_ft]

## 2. Model Re-Training

Fine-tuning involves continuing the training of an already trained model on new or existing data, typically with a much smaller learning rate. This allows the model to adjust its weights subtly without forgetting the extensive knowledge it gained during initial training.

### 2.1 Recompiling for Re-Training

A critical step in fine-tuning is to **recompile the model with a significantly smaller learning rate**. This prevents large updates to the weights that could destabilize the already learned features. A common practice is to use a learning rate that is 1/10th or 1/100th of the original learning rate. Here, a new learning rate of `0.0000436` is set.

We also recompile with `loss='mae'` (Mean Absolute Error) and include `PHM2008Score()` as a metric. This ensures that the fine-tuning process directly optimizes for the MAE and monitors the challenge-specific PHM08 score, which is crucial for evaluating RUL predictions.

Additionally, `EarlyStopping` and `ModelCheckpoint` callbacks are configured specifically for the fine-tuning phase:

* **`EarlyStopping`**: Monitors `val_phm_2008_score` (validation PHM 2008 score) and stops training if it doesn't improve for 10 consecutive epochs, restoring the best weights. This ensures we stop at the optimal point for the specific challenge metric.

* **`ModelCheckpoint`**: Saves the best model based on the `val_phm_2008_score` to a new file, `fine_tuned_lstm_model_v2.keras`, preserving the best fine-tuned version.

### 2.2 Re-Training the Model

The `fine_tune_model` is now trained on the `X_train` and `y_train` data. Although the model is already trained, this step allows it to further optimize its weights based on the specified fine-tuning parameters and callbacks.

* **`epochs=100`**: A maximum of 100 epochs is set, but `EarlyStopping` will likely halt training sooner if the `val_phm_2008_score` does not improve.

* **`validation_data=0.2`**: A validation split of 20% is used to monitor performance on unseen data during fine-tuning, guiding the `EarlyStopping` and `ModelCheckpoint` callbacks.

* **`batch_size=2048`**: A larger batch size is used, which can sometimes speed up training for fine-tuning, though it might require careful tuning.

The output will show the training progress, including loss and metrics for both training and validation sets, indicating how the model adapts during the fine-tuning process.

In [ ]:
# --- Train the best model with optimal hyperparameters ---
print("\nTraining the best model with re_train_data...")

# Train the final model (you might use more epochs here)
history = fine_tune_model.fit(
    X_train,y_train,
    epochs=100, # Train for more epochs
    validation_data=0.2,
    callbacks=[callbacks_list_ft], # Use early stopping again
    batch_size=2048,
    verbose=1
)

print("\nFinal model training complete!")

## 3. Saving the Re-Trained Model

After the Re-training process is complete, the final `Re-trained model` (which has its weights restored to the best performing epoch during fine-tuning) is saved.

This saved model (`model/final_model.keras`) represents the updated, optimized version of your predictive maintenance solution. It can now be used for inference on new, real-world data, benefiting from both the initial extensive training and the targeted fine-tuning.

In [ ]:
# Save the model in .keras format
model_save_path_keras = "model/final_model.keras"
fine_tune_model.save(model_save_path_keras)
print(f"Final model saved in .keras format to: {model_save_path_keras}")

## 4. Conclusion

This notebook successfully demonstrates a practical approach to **model retraining and fine-tuning** for predictive maintenance. By loading a pre-trained model and continuing its training with a carefully chosen learning rate and relevant callbacks, we can efficiently adapt and improve the model's performance. This process is crucial for maintaining model relevance and accuracy in dynamic real-world environments where new data becomes available or conditions change.

Fine-tuning allows for:
* **Leveraging Pre-trained Knowledge:** Avoiding training from scratch, saving computational resources and time.
* **Adapting to New Data:** Incorporating new degradation patterns or operational conditions.
* **Refining Performance:** Achieving incremental improvements on specific metrics.

This showcases an important aspect of a robust machine learning lifecycle, demonstrating your ability to deploy and maintain models effectively.